<a href="https://colab.research.google.com/github/MigguuelT/Miguel-Torikachvili/blob/main/Entrevistador_Virtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install google-genai

In [ ]:
import google.generativeai as genai
from IPython.display import display, Markdown
import ipywidgets as widgets
from ipywidgets import Layout
import os  # Importe o módulo os
from google.colab import userdata


# Configure a chave da API (substitua pela sua chave real)
# genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

# os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# Seleciona o modelo Gemini Pro
model = genai.GenerativeModel('gemini-2.0-flash')

# Define os widgets de entrada
vaga_input = widgets.Text(description="Vaga Desejada:", layout=Layout(width='50%'))
area_input = widgets.Text(description="Área de Atuação:", layout=Layout(width='50%'))
tom_input = widgets.Dropdown(
    options=['formal', 'informal', 'profissional e amigável', 'direto e objetivo'],
    description="Tom da Entrevista:",
    layout=Layout(width='50%')
)
conhecimentos_input = widgets.Text(description="Conhecimentos Específicos:", layout=Layout(width='50%'))
enviar_config_button = widgets.Button(description="Configurar Entrevista")
pergunta_output = widgets.Output()
resposta_input = widgets.Textarea(description="Sua Resposta:", layout=Layout(width='80%', height='150px'))
enviar_resposta_button = widgets.Button(description="Enviar Resposta")
feedback_output = widgets.Output()
historico_output = widgets.Output()
fim_entrevista = False
respostas_usuario = []
perguntas_realizadas = []

system_instruction_base = "Você é um entrevistador para a vaga de {vaga} na área de {area}. A entrevista deve ter um tom {tom}. Faça perguntas típicas para esta função, incluindo perguntas comportamentais focadas em trabalho em equipe e resolução de problemas, além de algumas perguntas sobre o conhecimento de {conhecimentos} e o interesse do candidato pela empresa. Por favor, analise a minha resposta à cada pergunta. Forneça feedback sobre a clareza da minha resposta, o uso de exemplos e como eu poderia ter respondido de forma mais eficaz, talvez usando a metodologia STAR."
system_instruction = ""
historico_conversa = []

def format_markdown(md_string):
    display(Markdown(md_string))

def configurar_entrevista(b):
    global system_instruction, fim_entrevista, respostas_usuario, perguntas_realizadas
    vaga = vaga_input.value
    area = area_input.value
    tom = tom_input.value
    conhecimentos = conhecimentos_input.value

    system_instruction = system_instruction_base.format(vaga=vaga, area=area, tom=tom, conhecimentos=conhecimentos)

    fim_entrevista = False
    respostas_usuario = []
    perguntas_realizadas = []
    historico_conversa.clear()
    with historico_output:
        historico_output.clear_output()
    with pergunta_output:
        pergunta_output.clear_output()
    with feedback_output:
        feedback_output.clear_output()

    with pergunta_output:
        format_markdown("**Entrevista configurada. Clique em 'Próxima Pergunta' para começar.**")

def finalizar_entrevista():
    global respostas_usuario, perguntas_realizadas, system_instruction
    feedback_total_prompt = f"{system_instruction}\n\nRespostas do candidato:\n"
    for i, resposta in enumerate(respostas_usuario):
        feedback_total_prompt += f"Pergunta {i+1}: {perguntas_realizadas[i]}\nResposta: {resposta}\n\n"
    feedback_total_prompt += "Forneça um feedback geral sobre o desempenho do candidato na entrevista, destacando os pontos fortes e as áreas de melhoria."

    try:
        response = model.generate_content(feedback_total_prompt)
        feedback_total = response.text

        with feedback_output:
            feedback_output.clear_output()
            format_markdown(f"**Entrevista Encerrada. Feedback Total:**\n{feedback_total}")

    except Exception as e:
        with feedback_output:
            feedback_output.clear_output()
            format_markdown(f"**Erro ao gerar feedback total:** {e}")

def enviar_resposta(b):
    global system_instruction, fim_entrevista, respostas_usuario, perguntas_realizadas
    resposta_usuario = resposta_input.value.lower()
    resposta_input.value = ""

    if resposta_usuario == "fim":
        fim_entrevista = True
        finalizar_entrevista()
        return

    if not system_instruction:
        with feedback_output:
            feedback_output.clear_output()
            format_markdown("**Por favor, configure a entrevista primeiro.**")
        return

    prompt_com_resposta = f"{system_instruction}\n\nMinha resposta foi: {resposta_usuario}"

    try:
        response = model.generate_content(prompt_com_resposta)
        feedback = response.text

        respostas_usuario.append(resposta_usuario)
        historico_conversa.append(f"**Você:** {resposta_usuario}")
        historico_conversa.append(f"**Feedback:** {feedback}")
        atualizar_historico()

        with feedback_output:
            feedback_output.clear_output()
            format_markdown(f"**Feedback:**\n{feedback}")

        if not fim_entrevista:
            gerar_proxima_pergunta()

    except Exception as e:
        with feedback_output:
            feedback_output.clear_output()
            format_markdown(f"**Erro ao gerar feedback:** {e}")

def gerar_proxima_pergunta():
    global system_instruction, fim_entrevista, perguntas_realizadas

    if fim_entrevista:
        with pergunta_output:
            pergunta_output.clear_output()
            format_markdown("**A entrevista foi encerrada.**")
        return

    if not system_instruction:
        with pergunta_output:
            pergunta_output.clear_output()
            format_markdown("**Por favor, configure a entrevista primeiro.**")
        return

    prompt_pergunta = f"{system_instruction}\n\nFaça a próxima pergunta."

    try:
        response = model.generate_content(prompt_pergunta)
        pergunta = response.text

        perguntas_realizadas.append(pergunta)
        historico_conversa.append(f"**Entrevistador:** {pergunta}")
        atualizar_historico()

        with pergunta_output:
            pergunta_output.clear_output()
            format_markdown(f"**Entrevistador:**\n{pergunta}")

    except Exception as e:
        with pergunta_output:
            pergunta_output.clear_output()
            format_markdown(f"**Erro ao gerar pergunta:** {e}")

def atualizar_historico():
    with historico_output:
        historico_output.clear_output()
        for item in historico_conversa:
            format_markdown(f"{item}\n\n")

proxima_pergunta_button = widgets.Button(description="Próxima Pergunta")
proxima_pergunta_button.on_click(lambda b: gerar_proxima_pergunta())
enviar_config_button.on_click(configurar_entrevista)
enviar_resposta_button.on_click(enviar_resposta)

# Exibe os widgets
display(vaga_input)
display(area_input)
display(tom_input)
display(conhecimentos_input)
display(enviar_config_button)
display(pergunta_output)
display(resposta_input)
display(enviar_resposta_button)
display(feedback_output)
display(widgets.Label(value="Histórico da Conversa:"))
display(historico_output)
display(proxima_pergunta_button)